# Understanding the Problem

In [ ]:
class KeyValuePair {
    constructor (public key: string, public value: string) {}
}

In the example above, We can only create key-value pairs where both the key and value are of type `string`. If we need to support other types, one option is to use the `any` type. However, using `any` removes type safety, which can lead to runtime errors and makes the code less robust. Another option is to create separate versions of the class for each data type, like `KeyValuePairNumber`, `KeyValuePairBoolean`, etc. But this leads to redundant code and is not scalable.

A better solution is to use **generics**, which allow us to define the class in a type-safe and reusable way.

# Generic Classes

In [ ]:
class KeyValuePair<T> {
    constructor (public key: T, public value: string) {}
}

let pair = new KeyValuePair<string>('name', 'John Doe');

In [ ]:
class KeyValuePair<T, V>{
    constructor (public key: T, public value: V) {}
}

let pair2 = new KeyValuePair<string, number>('age', 30);

If we **don't explicitly pass the types**, TypeScript is smart enough to **automatically infer the types** from the values we provide:

In [ ]:
class KeyValuePair<T, V>{
    constructor (public key: T, public value: V) {}
}

let pair3 = new KeyValuePair('isPublished', false);

# Generic Functions

In [ ]:
// A generic function that takes a value of any type T and returns it wrapped inside an array
function wrapInArray<T>(value: T){
    return [value];
}

// let number = wrapInArray<string>('42');
let number = wrapInArray(42);

This function can also be defined inside a class

In [ ]:
class ArrayUtils{
    wrapInArray<T>(value: T){
        return [value];
    }
}

let utils = new ArrayUtils();
let number2 = utils.wrapInArray(42);

# Generic Interfaces

In [ ]:
// ex -: returning data from an API
// http://mywebsite.com/users
// http://mywebsite.com/products

interface Result<T> {
    data: T | null;
    error: string | null;
}

function fetch<T>(url : string) : Result<T>{
    return {data:null, error: null};
}

interface User {
    username: string;
}

interface Product {
    title: string;
}


let result1 = fetch<User>('http://mywebsite.com/users');
result1.data?.username; // data will be null or User object

let result2 = fetch<Product>('http://mywebsite.com/products');
result2.data?.title; // data will be null or Product object

Explanation -->

1. fetch<User>('http://mywebsite.com/users') calls

2. The return type of data should be `Result<User>`

3. `Result<User>` have two elemets
        - data : <User> | null
        - error : string | null

4. Since `data` is either User object or null, We can access properties of `User` objects

# Generic Constraints

In [ ]:
// Constraint by type

function echo<T extends string | number>(value: T): T {  
    // The type T must be either string or number — no other types are allowed
    return value;
}

echo(1);

In [ ]:
// Constraint by the shape of an object

function echo<T extends { name:string } >(value: T): T {
    // T must be an object type that has at least a name property of type string.
    return value;
}

echo({name: 'John Doe'});

In [ ]:
// Constraint by an interface

interface Person {
    name : string;
}

function echo<T extends Person>(value: T): T {
    // T must be (or extend from) the Person type
    return value;
}

In [ ]:
// Constraint by a class

class Person {
    constructor(public name: string) {}
}

class Customer extends Person {
}

function echo<T extends Person>(value: T): T {
    // T must be the Person class or any class that inherits from Person
    return value;
}

echo(new Person('John Doe'));
echo(new Customer('Jane Doe')); 

# Extending Generic Classes

When extending a generic class, We have 3 options.

In [ ]:
// Example -: In an e-commerce application, we might have categories like Products, Orders, and Shopping Carts.

interface Product {
    name : string;
    price: number;
}

class Store<T>{
    protected _objects : T[] = [];

    add(object: T) : void {
        this._objects.push(object);
    }
}



// Option 1 - Pass on the generic type parameter
class CompressibleStore<T> extends Store<T>{
    compress() {}
}

let store = new CompressibleStore<Product>();
store.add({name: 'Laptop', price: 1000});
store.compress();



// Option 2 - Restrict the generic type parameter
class SearchableStore<T extends {name:string}> extends Store<T>{  // T must be a type that includes at least a name: string property
    find(name : string) : T | undefined {
        return this._objects.find(obj => obj.name === name );
    }
}

let searchableStore = new SearchableStore<Product>();
searchableStore.add({ name: 'Laptop', price: 1000 });



// Option 3 - Fix the generic type parameter
class ProductStore extends Store<Product> {
    filterByCategory(category : string) : Product[] {
        return [];
    }
}

let productStore = new ProductStore();
productStore.add({ name: 'Laptop', price: 1000 });
productStore.filterByCategory('Electronics');

# Keyof Operator

The `keyof` operator returns a union of the keys (property names) of a given type or interface.

In [ ]:
interface Product {
    name : string;
    price: number;
}

class NewStore<T>{
    protected _objects : T[] = [];

    add(object: T) : void {
        this._objects.push(object);
    }

    // T is Product
    // keyof T => 'name' | 'price'
    find(property: keyof T, value: unknown): T | undefined {
        return this._objects.find(obj => obj[property] === value);
    }
}


let mystore = new NewStore<Product>();
mystore.add({name: 'Laptop', price: 1000});

mystore.find('name', 'Laptop'); // Returns the product with name 'Laptop'
mystore.find('price', 1000); // Returns the product with price 1000
mystore.find('unKnownProperty', 100); // This will **cause a TypeScript error**!

If you use a plain `string` type instead, any string can be passed—including invalid property names—which can cause errors or crashes when the program runs.

Using `keyof T` ensures that only valid property names of the type `T` can be passed to the `find` method, so if you try to use a wrong property name like `'unKnownExistingParameter'`, TypeScript will show an error and prevent the program from crashing at runtime. 

# Type Mapping

Using Type Mapping, we **can create new types by transforming each property of an existing type**.

For example, with mapped types, you can:

- Make all properties readonly
- Make all properties optional
- Change property types
- Create filters or pick certain keys dynamically

##### Example 1 -> Make all Properties readOnly

In [ ]:
interface Product {
    name : string;
    price: number;
}

type ReadOnlyProduct = {
    // Index signature
    // keyof
    readonly [K in keyof Product]: Product[K];
}

let product1 : ReadOnlyProduct = {
    name : 'a',
    price: 100
};

product1.name = 'b'; // This will cause a TypeScript error because name is read-only

##### Example 2 -> Make all Properties readOnly ( in a generic class )

In [ ]:
interface Product {
    name : string;
    price: number;
}

type ReadOnly<T> = {
    readonly [K in keyof T]: T[K];
}

let product2 : ReadOnly<Product> = {
    name : 'a',
    price: 100
};

##### Example 3 -> Make all Properties Optional

In [ ]:
interface Product {
    name : string;
    price: number;
}

type Optional<T> = {
 [K in keyof T]?: T[K];
}

##### Example 4 -> Make all Properties Nullable

In [ ]:
interface Product {
    name : string;
    price: number;
}

type Nullable<T> = {
 [K in keyof T]?: T[K] | null;
}

All these type mapping utilities are already built into TypeScript, so we can use them directly. We can also review and customize them as needed.